In [5]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 
from matplotlib import pyplot as plt
import numpy as np

In [6]:
# Download the data from kaggle and store it the data folder
# https://www.kaggle.com/new-york-city/nyc-parking-tickets#Parking_Violations_Issued_-_Fiscal_Year_2017.csv

In [13]:
file_path = './data/nyc-parking-tickets/*2017.csv'
df = dd.read_csv(file_path)

In [14]:
df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=33,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int64,object,object,object,int64,int64,int64,int64,float64,int64,int64,int64,object,object,object,object,object,object,object,object,object,int64,int64,object,object,object,object,object,object,float64,int64,object,int64,object,object,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
dtypes = {
 'Date First Observed': np.str,
 'Days Parking In Effect    ': np.str,
 'Double Parking Violation': np.str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': np.str,
 'House Number': np.str,
 'Hydrant Violation': np.str,
 'Intersecting Street': np.str,
 'Issue Date': np.str,
 'Issuer Code': np.float32,
 'Issuer Command': np.str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': np.str,
 'Issuing Agency': np.str,
 'Law Section': np.float32,
 'Meter Number': np.str,
 'No Standing or Stopping Violation': np.str,
 'Plate ID': np.str,
 'Plate Type': np.str,
 'Registration State': np.str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': np.str,
 'Sub Division': np.str,
 'Summons Number': np.uint32,
 'Time First Observed': np.str,
 'To Hours In Effect': np.str,
 'Unregistered Vehicle?': np.str,
 'Vehicle Body Type': np.str,
 'Vehicle Color': np.str,
 'Vehicle Expiration Date': np.str,
 'Vehicle Make': np.str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': np.str,
 'Violation Description': np.str,
 'Violation In Front Of Or Opposite': np.str,
 'Violation Legal Code': np.str,
 'Violation Location': np.str,
 'Violation Post Code': np.str,
 'Violation Precinct': np.float32,
 'Violation Time': np.str
}

nyc_df = dd.read_csv('data/nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())


Dask dataframes look and feel like Pandas dataframes but they run on the same infrastructure that powers dask.delayed

### Selection and filtering

In [10]:
#Column selection
nyc_df['Issue Date'].head(n=1)

0    07/10/2016
Name: Issue Date, dtype: object

In [11]:
#Columns selection
nyc_df[['Issue Date', 'Plate Type']].head(n=2)

,Issue Date,Plate Type
0,07/10/2016,PAS
1,07/08/2016,PAS


In [12]:
nyc_df.loc[:200].head(n=2)

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,797502185,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001.0,NaN,0.0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,797484362,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001.0,NaN,0.0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN


In [39]:
temp_rows = nyc_df.loc[:200].head(n=100) #returns a pandas dataframe

In [40]:
len(temp_rows)

100

### data cleaning process

#### Remove rows and columns

In [41]:
missing_count = temp_rows.isnull().sum() #missing_values is a series

In [42]:
percent_missing = ((missing_count / temp_rows.index.size) * 100)

In [43]:
percent_missing

Summons Number                         0.0
Plate ID                               0.0
Registration State                     0.0
Plate Type                             0.0
Issue Date                             0.0
Violation Code                         0.0
Vehicle Body Type                      0.0
Vehicle Make                           0.0
Issuing Agency                         0.0
Street Code1                           0.0
Street Code2                           0.0
Street Code3                           0.0
Vehicle Expiration Date                0.0
Violation Location                    20.0
Violation Precinct                     0.0
Issuer Precinct                        0.0
Issuer Code                            0.0
Issuer Command                        20.0
Issuer Squad                          20.0
Violation Time                         0.0
Time First Observed                   93.0
Violation County                       0.0
Violation In Front Of Or Opposite     21.0
House Numbe

In [44]:
columns_to_drop = list(percent_missing[percent_missing >= 50].index)

In [45]:
columns_to_drop

['Time First Observed',
 'Intersecting Street',
 'Violation Legal Code',
 'Unregistered Vehicle?',
 'Meter Number',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']

In [46]:
nyc_df_stage1 = nyc_df.drop(columns_to_drop, axis=1)

In [47]:
nyc_df_stage1.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
0,797502185,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,1111.0,D,NaN,NaN,NaN,GY,2001.0,0.0,NaN,FAILURE TO STOP AT RED LIGHT
1,797484362,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,1111.0,D,NaN,NaN,NaN,GY,2001.0,0.0,NaN,FAILURE TO STOP AT RED LIGHT
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,1111.0,C,NaN,NaN,NaN,BK,2004.0,0.0,NaN,BUS LANE VIOLATION
3,4183662532,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,408.0,l2,Y,0700A,0700P,WH,2007.0,0.0,04,47-Double PKG-Midtown
4,3573333014,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,408.0,h1,Y,0700A,0700P,WHITE,2007.0,0.0,31 6,69-Failure to Disp Muni Recpt


#### Imputing missing values

In [17]:
# Fill the missing Violation County with the most common one)

#  value_counts method counts the unique occurrences of data
count_of_violation_county = nyc_df_stage1['Violation County'].value_counts().compute()

In [18]:
type(count_of_violation_county)

pandas.core.series.Series

In [19]:
most_common_county = count_of_violation_county.sort_values(ascending=False).index[0]

In [48]:
most_common_county

'NY'

In [21]:
nyc_df_stage2 = nyc_df_stage1.fillna({'Violation County': most_common_county})

#### Dropping rows with missing value

In [22]:
drop_rows = list(percent_missing[(percent_missing > 0) & (percent_missing < 10)].index)
#Subset argument specifies which columns to check for null values
nyc_df_stage3 = nyc_df_stage2.dropna(subset=drop_rows).compute() 